<style>
pre > code {
    background-color: #3A3960 !important;
    padding: 10px;
    display: block;
    border-radius: 5px;
    border: 1px solid #ccc;
    overflow-x: auto;
}
</style>

# 4.4 Grundlegende FastAPI Konzepte: Database, Authentication, Authorization, Hashing Passwords

Nachdem wir viele Grundlagen von FastAPI anhand des Bücherbeispiels kennen gelernt haben, wollen wir weitere Konzepte an einer TO-DO App lernen. Die nächsten Schwerpunkte sind:
- Datenbanken: Wir arbeiten mit einer vollständigen SQL-Datenbank und lernen drei verschiedene Produktionsdatenbanken kennen:
  - SQLite (eingebettete Datenbank, einfacher Einstieg)
  - PostgreSQL & MySQL (leistungsfähige Datenbanken für echte Anwendungen)
- Benutzerauthentifizierung mit JWT:
  - Nutzer können sich mit Benutzernamen & Passwort registrieren und anmelden.
  - Passwörter werden gehasht, um die Sicherheit zu gewährleisten.
- Autorisierung & Rollenmanagement:
  - Benutzer erhalten unterschiedliche Rollen (z. B. Admin).
  - Admins haben Zugriff auf spezielle API-Endpunkte, die andere nicht nutzen können.
- Architektur & Sicherheit:
  - Unsere Webseite kommuniziert mit dem FastAPI-Server.
  - FastAPI verarbeitet Anfragen, überprüft Authentifizierung & Autorisierung und greift auf die Datenbank zu.
  - Wir implementieren moderne Sicherheitsmaßnahmen für eine professionelle API.

Als erstes erstellen wir eine Datei mit dem Namen "database.py". Anschließend installieren wir SQL-Alchemy:
```
pip install SQLAlchemy
```

SQLAlchemy ist eine Python-Bibliothek, die uns hilft, mit Datenbanken zu arbeiten.
Anstatt komplizierte SQL-Befehle zu schreiben, kann man SQLAlchemy für eine einfache und strukturierte Art der Datenbankkommunikation nutzen. Einige Vorteile sind:
- Man kann Datenbanktabellen als Python-Objekte behandeln.
- Es erleichtert die Verwaltung von Datenbanken (Erstellen, Ändern, Abfragen von Daten).
- Es funktioniert mit vielen Datenbanken (z. B. SQLite, MySQL, PostgreSQL).
- Es gibt dir zwei Möglichkeiten zu arbeiten: Mit reinem SQL oder mit ORM (Object-Relational Mapping).

ORM bedeutet, dass man Datenbanktabellen wie Python-Klassen behandelt.
Statt direkt SQL zu schreiben, nutzt man Python-Objekte, um Daten zu speichern, zu bearbeiten und abzurufen. Dank ORM hat man also folgende Vorteile:
- Man muss kein SQL schreiben, man arbeitet mit Python-Klassen und Objekten.
- Es macht den Code lesbarer und einfacher.
- Es ist sicherer, weil du dich nicht um SQL-Injections kümmern musst.
- Es funktioniert mit verschiedenen Datenbanktypen (du kannst leicht von SQLite zu PostgreSQL wechseln).

Als erstes erstellen wir eine Konstente:
```python
SQLALCHEMY_DATABASE_URL = "sqlite:///./todos.db"
```

Diese URL wird verwendet, um einen Speicherort für die Datenbank anzugeben:
- "sqlite:///" gibt an, dass SQLite als Datenbank-Engine verwendet wird
- "./" bedeutet, dass die Datei im aktuellen Verzeichnis gespeichert wird.
- "todos.db" ist der Dateiname der SQLite-Datenbank.
- Allgemein "sqlite:///pfad/zur/datenbank.db"

Jetzt müssen wir von "sqlalchemy" die Funktion "create_engine" importieren:
- "create_engine" ist eine Funktion in SQLAlchemy, die eine Datenbank-Engine erstellt.
- Die Engine ist die Schnittstelle zwischen SQLAlchemy und der Datenbank.
- Sie verwaltet Verbindungen zur Datenbank und führt SQL-Befehle aus.

```python
from sqlalchemy import create_engine

SQLALCHEMY_DATABASE_URL = "sqlite:///./todos.db"

engine = create_engine(SQLALCHEMY_DATABASE_URL, connect_args={"check_same_thread": False})
```

Was genau macht eigentlich "connect_args={"check_same_thread": False}"?<br>
Dadurch erlaubt man mehreren Teilen des Programms, gleichzeitig mit der SQLite-Datenbank zu arbeiten. Wenn wir sagen "ein Teil des Programms", meinen wir eine Aufgabe oder eine Aktion, die das Programm gerade ausführt.
<br>
<br>
Warum ist diese Einstellung so wichtig?<br>
SQLite erlaubt normalerweise nur einem einzigen Teil des Programms, die Verbindung zur Datenbank zu benutzen. FastAPI arbeitet aber so, dass mehrere Teile des Programms gleichzeitig irgendwelche Dinge machen können. Ohne diese Einstellung würde SQLite einen Fehler ausgeben, weil es denkt: "Hey, jemand anderes benutzt meine Verbindung!" Diese Einstellung brauchst man nur für SQLite, nicht für größere Datenbanken wie PostgreSQL oder MySQL.
<br>
<br>
Stellt euch vor, man hat eine To-Do-App, bei der Benutzer Aufgaben speichern können.
Das Programm kann mehrere Dinge gleichzeitig tun, z. B.:
- Ein Benutzer speichert eine neue Aufgabe.
- Ein anderer Benutzer ruft seine gespeicherten Aufgaben ab.

Beide Aktionen passieren gleichzeitig, also gibt es zwei "Teile" des Programms, die auf die Datenbank zugreifen möchten. SQLite erlaubt standardmäßig nur einem einzigen Teil (z. B. nur dem Speichern oder nur dem Abrufen) die Datenbank zu benutzen.
Wenn FastAPI aber mehrere Dinge gleichzeitig macht, kann das zu einem Fehler führen.
<br>
<br>
Jetzt importieren wir noch sessionmaker. Es handelt sich um eine Funktion  in SQLAlchemy, mit der wir Datenbank-Sitzungen (Sessions) erstellen. Eine Session ist eine Art "Verbindung" zur Datenbank, mit der wir Daten abrufen, hinzufügen, aktualisieren oder löschen können.

```python
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

SQLALCHEMY_DATABASE_URL = "sqlite:///./todos.db"

engine = create_engine(SQLALCHEMY_DATABASE_URL, connect_args={"check_same_thread": False})
```

Jetzt erstellen wir eine Session Klasse, doe später genutzt wird, um eine Verbindung zur Datenbank herzustellen und Daten zu verwalten:

```python
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

SQLALCHEMY_DATABASE_URL = "sqlite:///./todos.db"

engine = create_engine(SQLALCHEMY_DATABASE_URL, connect_args={"check_same_thread": False})

SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
```

- "autocommit=False": Legt fest, dass Änderungen nicht automatisch gespeichert werden. Wir müssen "db.commit()" aufrufen, um Änderungen dauerhaft zu machen
- "autoflush=False": Verhindert, dass Daten automatisch in die Datenbank geschrieben werden, bevor eine Abfrage erfolgt. Stattdessen müssen wir "db.flush()" oder "db.commit()" aufrufen, um Änderungen zu speicher
- "bind=engine": Bedeutet, dass diese Session mit unserer Datenbank-Engine (engine) verbunden ist. Dadurch weiß SQLAlchemy, mit welcher Datenbank die Sitzung arbeiten soll.

Jetzt müssen wir noch "declarative_base" importieren, um eine Basisklasse für unsere Datenbank-Modelle zu erstellen. Jede Tabelle, die wir in unserer Datenbank haben wollen, wird als Python-Klasse definiert und von Base abgeleitet:

```python
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

SQLALCHEMY_DATABASE_URL = "sqlite:///./todos.db"

engine = create_engine(SQLALCHEMY_DATABASE_URL, connect_args={"check_same_thread": False})

SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

Base = declarative_base()
```

Jetzt erstellen wir die Datei "models.py". Diese Datei wird verwendet um sogenannte Datenbank-Modelle (Tabellen) zu definieren. Das hilft, den Code übersichtlich zu halten, indem alle Datenbank-Strukturen an einem Ort organisiert sind. Wir werden eine Tabelle mit dem Namen "Todos" haben. In ihr sind Spalten wie z.B. "ID" oder "Titel" enthalten:

```python
from database import Base
from sqlalchemy import Column, Integer, String, Boolean

class Todos(Base):
    __tablename__ = "todos"
```

Dieser Code definiert also eine Tabelle namens "todo" für eine Datenbank mit SQLAlchemy ("class Todos(Base):"). Wie man sieht erben hier alle Klassen bzw. Tabellen von der Grundklasse "Base", welche wir in der "database.py" Datei erstellt hatten.
<br>
<br>
Außerdem haben wir noch vier Klassen importiert:
- "Column" wird dan nverwendet um Spalten in der Tabelle zu definieren.
- "Integer" wird dann verwendet, um zu zeigen, dass es sich um Ganzzahlen in einer Spalte handelt.
- "String" wird ebenfalls dann verwendet, um zu zeigen dass es sich in der Spalte um Einträge vom Datentyp String (Zeichenkette) handelt.
- - "Boolean" wird ebenfalls dann verwendet, um zu zeigen dass es sich in der Spalte um Einträge vom Datentyp Boolean (Wahrheitswerte) handelt.

Die Zeile `__tablename__ = "todos"` sagt dass die Tabelle in der Datenbank "todos" heißen soll.
<br>
<br>
Nun legen wir die einzelnen Spalten mit den dazugehörigen Datentypen an:

```python
from database import Base
from sqlalchemy import Column, Integer, String, Boolean

class Todos(Base):
    __tablename__ = "todos"
    
    id = Column(Integer, primary_key=True, index=True)
    title = Column(String)
    description = Column(String)
    priority = Column(Integer)
    complete = Column(Boolean, default=False)
```

Die Zeile:
- "id = Column(Integer, primary_key=True, index=True)" erstellt eine Spalte mit dem Namen "id". Mit "index=True" sagen wir aus dass die Spalte schneller durchsucht werden kann. Man kann sich den "index", als eine Art Inhaltsverzeichnis eines Buchen sich vorstellen, damit können wir schneller Daten finden. Durch "primary_key=True", sagen wir dass die Spalte als Primärschlüssel behandelt werden soll. Jede Zeile in einer Tabelle braucht eine eindeutige Kennung (ID), um sie zu identifizieren. Diese ID darf sich nicht wiederholen und ist für jede Zeile einzigartig.
- "title = Column(String)" erstellt eine normale Spalte vom Datentyp String.
- "description = Column(String)" erstellt eine normale Spalte vom DAtentyp String
- "priority = Column(Integer)" erstellt eine normale Spalte vom Datentyp Integer.
- " complete = Column(Boolean, default=False)" erstellt eine normale Spalte vom Datentyp Boolean, wobei durch "default" wir von Anfang an einen Wert von "False" jeder Zelle zuweisen.


















